#Import Libraries and Mount Drive


In [1]:
import os
import sys
import shutil 
import glob

import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2 as cv


import torch
import torchvision
from time import time
from torchvision import datasets, transforms
from torchvision.io import read_image
from torch import nn, optim
from torch.utils.data import Dataset
print(torch.__version__)

1.8.1+cu101


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Create Directories to train the model

In [ ]:
parent_dir_j = "/content/drive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)"
parent_dir = "/content/drive/MyDrive/LWIR_to_RGB"
#os.mkdir(parent_dir)

In [5]:
cd '/content/drive/MyDrive/LWIR_to_RGB'

/content/drive/.shortcut-targets-by-id/12Sz3Xm5GOveoM4c19uxdsdFReUqsU8dW/LWIR_to_RGB


In [ ]:
!ls '/content/drive/MyDrive/LWIR_to_RGB'

A  B


In [ ]:
dirc = "A"
path_lwir = os.path.join(parent_dir,dirc)
#os.mkdir(path_lwir)

dirc = "B"
path_rgb = os.path.join(parent_dir,dirc)
#os.mkdir(path_rgb)

In [ ]:
path_lwir_train = os.path.join(path_lwir,"train" )
#os.mkdir(path_lwir_train)

path_lwir_val = os.path.join(path_lwir, "val")
#os.mkdir(path_lwir_val)

path_lwir_test = os.path.join(path_lwir, "test")
#os.mkdir(path_lwir_test)

path_rgb_train = os.path.join(path_rgb, "train")
#os.mkdir(path_rgb_train)

path_rgb_val = os.path.join(path_rgb, "val")
#os.mkdir(path_rgb_val)

path_rgb_test = os.path.join(path_rgb, "test")
#os.mkdir(path_rgb_test)


#Transfer images from dataset to train, val, test in A (LWIR) and B (RGB)


In [ ]:
# Transfering uncovered images for train and val data

cover_type = 'uncover'
count = 1
for i in range(1,21):
  if i < 10:
      data_name = "0000"+str(i)              
  else:
      data_name = "000"+str(i) 
  
  #Destination path
  if i<=15: 
    destination_lwir = path_lwir_train
    destination_rgb = path_rgb_train
    if i==15:
      print("Moving last subject in Training")
  elif (i>15 and i<=20):
    destination_lwir = path_lwir_val
    destination_rgb = path_rgb_val
    if i==20:
      print("Moving last subject in Val")

  for j in range(1,46):
    if j < 10:
        img_name = "00000"+str(j)
    else:
        img_name = "0000"+str(j)  


    #Source LWIR
    source = '/content/drive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)/train/' + data_name +'/IR/'+cover_type+'/image_'+img_name +'.png'
    #im_ir = cv.imread(source)
    #im_ir = cv.resize(im_ir, (576, 1024), interpolation = cv.INTER_AREA)
    #print(im_ir.shape)
    #cv.imwrite(os.path.join(destination_lwir,str(count)+'.png' ), im_ir)
    shutil.copy(source, destination_lwir)
    os.rename(os.path.join(destination_lwir,'image_'+img_name +'.png' ),os.path.join(destination_lwir,str(count)+'.png' ))
    #count+=1

    #Source RGB
    source = '/content/drive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)/train/' + data_name +'/RGB/'+cover_type+'/image_'+img_name +'.png'
    im_rgb = cv.imread(source)
    im_rgb = cv.resize(im_rgb, (120,160), interpolation = cv.INTER_AREA)
    #print(im_rgb.shape)
    cv.imwrite(os.path.join(destination_rgb,str(count)+'.png' ), im_rgb)
    #shutil.copy(source, destination_rgb)
    #os.rename(os.path.join(destination_rgb,'image_'+img_name +'.png' ),os.path.join(destination_rgb,str(count)+'.png' ))
    count+=1
    #break
  #break
    

Moving last subject in Training
Moving last subject in Val


In [ ]:
# Transfering covered images for test data

cover_type = 'cover2'    #change this for thin cover => "cover1" and thick cover => "cover2"
#count = 1
for i in range(71,73):      # any subjects in range 31-55 for thin cover and 56-80 for thick cover 
  if i < 10:
      data_name = "0000"+str(i)              
  else:
      data_name = "000"+str(i) 
  
  #Destination path

  
  destination_lwir = path_lwir_test
  destination_rgb = path_rgb_test 
  if i==32:
    print("Moving last subject in Testing")

  for j in range(1,46):
    if j < 10:
        img_name = "00000"+str(j)
    else:
        img_name = "0000"+str(j)  


    #Source LWIR
    source = '/content/drive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)/train/' + data_name +'/IR/'+cover_type+'/image_'+img_name +'.png'
    #im_ir = cv.imread(source)
    #im_ir = cv.resize(im_ir, (576, 1024), interpolation = cv.INTER_AREA)
    #print(im_ir.shape)
    #cv.imwrite(os.path.join(destination_lwir,str(count)+'.png' ), im_ir)
    shutil.copy(source, destination_lwir)
    os.rename(os.path.join(destination_lwir,'image_'+img_name +'.png' ),os.path.join(destination_lwir,str(count)+'.png' ))
    #count+=1

    #Source RGB
    source = '/content/drive/MyDrive/In-Bed-Human-Pose-Estimation(VIP-CUP)/train/' + data_name +'/RGB/'+cover_type+'/image_'+img_name +'.png'
    im_rgb = cv.imread(source)
    im_rgb = cv.resize(im_rgb, (120,160), interpolation = cv.INTER_AREA)
    #print(im_rgb.shape)
    cv.imwrite(os.path.join(destination_rgb,str(count)+'.png' ), im_rgb)
    #shutil.copy(source, destination_rgb)
    #os.rename(os.path.join(destination_rgb,'image_'+img_name +'.png' ),os.path.join(destination_rgb,str(count)+'.png' ))
    count+=1
    #break
  #break
    

# Install 
 from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2337, done.
remote: Total 2337 (delta 0), reused 0 (delta 0), pack-reused 2337
Receiving objects: 100% (2337/2337), 8.09 MiB | 14.00 MiB/s, done.
Resolving deltas: 100% (1499/1499), done.


In [ ]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 686kB 13.6MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp37-none-any.whl size=655251 sha256=d67f40c6b048091f2552ff9cea5062f2d97541c32a25203a95275e6966840db3
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5712 sha256=b89376b22514447de8f38ad9468b1981f9a6f483e603d420b0b47ea5dbb6066e
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile


# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_pix2pix_dataset.sh [cityscapes, night2day, edges2handbags, edges2shoes, facades, maps]`

Or use your own dataset by creating the appropriate folders and adding in the images. Follow the instructions [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/datasets.md#pix2pix-datasets).

In [ ]:
#!bash ./datasets/download_pix2pix_dataset.sh facades

In [ ]:
cd '/content/drive/MyDrive/LWIR_to_RGB/pytorch-CycleGAN-and-pix2pix'

/content/drive/.shortcut-targets-by-id/12Sz3Xm5GOveoM4c19uxdsdFReUqsU8dW/LWIR_to_RGB


Run this to create dataset used uncovered LWIR and RGB pairs in training and Validation, and covered pairs in test

In [ ]:
!python datasets/combine_A_and_B.py --fold_A /content/drive/MyDrive/LWIR_to_RGB/A  --fold_B /content/drive/MyDrive/LWIR_to_RGB/B --fold_AB /content/drive/MyDrive/LWIR_to_RGB

[fold_A] =  /content/drive/MyDrive/LWIR_to_RGB/A
[fold_B] =  /content/drive/MyDrive/LWIR_to_RGB/B
[fold_AB] =  /content/drive/MyDrive/LWIR_to_RGB
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  False
split = train, use 675/675 images
split = train, number of images = 675
split = val, use 225/225 images
split = val, number of images = 225
split = test, use 180/180 images
split = test, number of images = 180


#Training Pix2Pix

In [ ]:
cd '/content/drive/MyDrive/LWIR_to_RGB'

/content/drive/.shortcut-targets-by-id/12Sz3Xm5GOveoM4c19uxdsdFReUqsU8dW/LWIR_to_RGB


In [ ]:
!python /content/drive/MyDrive/LWIR_to_RGB/pytorch-CycleGAN-and-pix2pix/train.py --dataroot /content/drive/MyDrive/LWIR_to_RGB --name lwir_rgb_jat --model pix2pix --direction AtoB  --save_epoch_freq 400 --n_epochs 200 --n_epochs_decay 200 

Streaming output truncated to the last 5000 lines.
End of epoch 104 / 400 	 Time Taken: 45 sec
learning rate 0.0002000 -> 0.0002000
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/li

In [ ]:
ls '/content/drive/MyDrive/LWIR_to_RGB/checkpoints/lwir_rgb_jat/'

/content/drive/.shortcut-targets-by-id/12Sz3Xm5GOveoM4c19uxdsdFReUqsU8dW/LWIR_to_RGB


#Testing on trained model using covered images


In [ ]:
!python /content/drive/MyDrive/LWIR_to_RGB/pytorch-CycleGAN-and-pix2pix/test.py --dataroot /content/drive/MyDrive/LWIR_to_RGB  --direction AtoB --model pix2pix --name lwir_rgb_jat --num_test 180

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/LWIR_to_RGB	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                        

# Not necessary but kept for reference

## Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

## Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

## Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_fake_B.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_A.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_B.png')
plt.imshow(img)

## Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_pix2pix_model.sh [edges2shoes, sat2map, map2sat, facades_label2photo, and day2night]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [ ]:
!bash ./scripts/download_pix2pix_model.sh facades_label2photo

Note: available models are edges2shoes, sat2map, map2sat, facades_label2photo, and day2night
Specified [facades_label2photo]
for details.

--2021-06-08 15:05:45--  http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/facades_label2photo.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217704720 (208M)
Saving to: ‘./checkpoints/facades_label2photo_pretrained/latest_net_G.pth’

./checkpoints/facad 100%[===================>] 207.62M  1.17MB/s    in 96s     

2021-06-08 15:07:22 (2.17 MB/s) - ‘./checkpoints/facades_label2photo_pretrained/latest_net_G.pth’ saved [217704720/217704720]



In [ ]:
!python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./LWIR_to_RGB/A               	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              